In [6]:
import os
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [40]:
lst_sq_trains = ['MOT17-02-FRCNN', 'MOT17-04-FRCNN', 'MOT17-05-FRCNN', 'MOT17-09-FRCNN', 'MOT17-10-FRCNN', 'MOT17-11-FRCNN']
# lst_sq_trains = ['MOT17-02-DPM']
root_dir = './data/MOT17/train'
gt = 'gt/gt.txt'
save_path = './data/person_reid/train'
os.makedirs('./data/person_reid/train', exist_ok = True)


for idx, folder_name in enumerate(lst_sq_trains):
    with open(os.path.join(root_dir, folder_name, gt), 'r') as f:
        progress_bar = tqdm(f.readlines(), desc=f"Folder {folder_name}", leave=False)
        for line in progress_bar:
            img_name = '0'*(6-len(line.split(',')[0])) + line.split(',')[0] + '.jpg'
            img_id = line.split(',')[1]
            img = cv2.imread(f'{root_dir}/{folder_name}/img1/{img_name}')
            left = int(line.split(',')[2])
            top = int(line.split(',')[3])
            width = int(line.split(',')[4])
            height = int(line.split(',')[5])
            im_crop = img[top: top+height,left: left+width]
            try:
                cv2.imwrite(f'{save_path}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
            except:
                continue
        #     plt.imshow(im_crop)
        #     plt.show()
        #     break
        # break

In [6]:
import numpy as np
x = np.array([1, 2, 3, 4])
x[2:] = x[:2] + x[2:]
x

array([1, 2, 4, 6])

In [39]:
val_seq = ['MOT17-13-FRCNN']
root_dir = './data/MOT17/train'
gt = 'gt/gt.txt'
save_path_query = './data/person_reid/val/query'
save_path_gallery = './data/person_reid/val/gallery'
os.makedirs(save_path_query, exist_ok = True)
os.makedirs(save_path_gallery, exist_ok = True)

for idx, folder_name in enumerate(val_seq):
    with open(os.path.join(root_dir, folder_name, gt), 'r') as f:
        lines = f.readlines()
        idx_arr = np.arange(len(lines))
        idx_random_20_percent = np.random.choice(idx_arr, size=int(0.2 * len(idx_arr)), replace=False)
        progress_bar = tqdm(lines, desc=f"Folder {folder_name}", leave=False)

        for idx, line in enumerate(progress_bar):
            img_name = '{:06d}'.format(int(line.split(',')[0])) + '.jpg'
            img_id = line.split(',')[1]
            img = cv2.imread(f'{root_dir}/{folder_name}/img1/{img_name}')
            left = int(line.split(',')[2])
            top = int(line.split(',')[3])
            width = int(line.split(',')[4])
            height = int(line.split(',')[5])
            im_crop = img[top: top+height,left: left+width]
            
            if idx in idx_random_20_percent:
                try:
                    cv2.imwrite(f'{save_path_query}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
                except:
                    continue
            else:
                try:
                    cv2.imwrite(f'{save_path_gallery}/{folder_name.split("-")[1]}_{img_name[:-4]}_{img_id}.jpg', im_crop)
                except:
                    continue

In [32]:
import argparse
import cv2
import os
from tqdm import tqdm
# limit the number of cpus used by high performance libraries
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
import torchvision.transforms as transforms
import sys
import platform
import numpy as np
from pathlib import Path
import torch
import torch.backends.cudnn as cudnn

from tracker.bytetrack.byte_tracker import BYTETracker
from tracker.strongsort.utils.parser import get_config


import logging
from ultralytics.nn.autobackend import AutoBackend  # For inference on various platform
from ultralytics.data.augment import LetterBox
from ultralytics.data.loaders import LoadImages, LoadStreams
from ultralytics.data.utils import IMG_FORMATS, VID_FORMATS
from ultralytics.utils import DEFAULT_CFG, LOGGER, SETTINGS, callbacks, colorstr, ops
from ultralytics.utils.checks import check_file, check_imgsz, check_imshow, print_args, check_requirements
from ultralytics.utils.files import increment_path
from ultralytics.utils.torch_utils import select_device
from ultralytics.utils.ops import Profile, non_max_suppression, scale_boxes, process_mask, process_mask_native
from ultralytics.utils.plotting import Annotator, colors, save_one_box
from tracker.get_trackers import create_tracker

In [33]:
source = "/home/son/Desktop/VIN/CV/project/data/MOT17/train/MOT17-02-DPM/img1"
imgsz = 640
vid_stride = 1

dataset = LoadImages(source, imgsz=imgsz, vid_stride=vid_stride)

In [34]:
yolo_weights = "weights/best.pt"
device = select_device(0)
dnn = False
half = True


model = AutoBackend(yolo_weights, device=device, dnn=dnn, fp16=half)
stride, names, pt = model.stride, model.names, model.pt

Ultralytics YOLOv8.0.184 🚀 Python-3.8.17 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3902MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


In [46]:
augment = False
visualize = False
transforms = getattr(model.model, 'transforms', None)

for frame_idx, batch in tqdm(enumerate(dataset)):
    path, im0, vid_cap, s = batch
    im0 = im0[0]
    im = LetterBox(imgsz, True, stride=stride)(image=im0)
    im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    im = np.ascontiguousarray(im)  # contiguous


    # visualize = increment_path(save_dir / Path(path[0]).stem, mkdir=True) if visualize else False


    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()
    im /= 255.0
    if len(im.shape) == 3:
        im = im[None]   # expand for batch dim
    print(im.shape)
    preds = model(im, augment=augment, visualize=visualize)

7it [00:00, 37.27it/s]

torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])


19it [00:00, 50.24it/s]

torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])


25it [00:00, 49.91it/s]

...

122it [00:02, 57.59it/s]

torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])


136it [00:02, 54.82it/s]

torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])
torch.Size([1, 3, 384, 640])


KeyboardInterrupt: 

In [29]:
!pip install -U ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/ae/38/eb300b64bab80039f72cb00dbc7e82c6478bf1acc89031247632fd504221/ultralytics-8.0.184-py3-none-any.whl.metadata
  Using cached ultralytics-8.0.184-py3-none-any.whl.metadata (28 kB)
Using cached ultralytics-8.0.184-py3-none-any.whl (618 kB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.20
    Uninstalling ultralytics-8.0.20:
      Successfully uninstalled ultralytics-8.0.20


In [20]:
from ultralytics.yolo.data.dataloaders.stream_loaders import LoadImages

WARNING ⚠️ Ultralytics settings reset to defaults. 
This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
You may view and update settings directly in '/home/son/.config/Ultralytics/settings.yaml'


In [22]:
transforms=getattr(model.model, 'transforms', None)
transforms

In [25]:
model = AutoBackend('/home/son/Desktop/VIN/CV/project/weights/yolov8x.pt', device=device, dnn=dnn, fp16=half)

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [27]:
transforms=getattr(model.model, 'transforms', None)
transforms